<a href="https://colab.research.google.com/github/AIsoroush/deep-learning-projects/blob/main/recommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

سروش تقدس – Movie Recommendation System

در این پروژه یک سیستم توصیه‌گر فیلم کاربر به کاربر (User-Based Collaborative Filtering) طراحی شد. داده‌ها شامل اطلاعات فیلم‌ها و امتیازهای کاربران بود. ابتدا داده‌ها پاکسازی و پردازش شدند و ستون‌های اضافی مثل ژانر و timestamp حذف شدند. سپس با استفاده از ضریب همبستگی پیرسون، کاربران مشابه با کاربر جدید شناسایی شدند. با محاسبه میانگین وزن‌دار امتیازها، فیلم‌هایی که بیشترین تناسب را با سلیقه کاربر جدید داشتند، به او پیشنهاد شد.

خلاصه نتایج:

سیستم قادر است ۲۰ فیلم برتر را با بیشترین تطابق سلیقه به کاربر پیشنهاد دهد.

روش پیشنهادی بر اساس تشابه واقعی سلیقه کاربران و نه صرفاً امتیاز متوسط فیلم‌ها عمل می‌کند.

قابلیت افزایش دقت با گسترش دیتاست و افزودن کاربران جدید وجود دارد.

In [3]:
# --- وارد کردن کتابخانه‌ها ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
%matplotlib inline

In [47]:
#دانلود فایل از گوگل درایو من برای دسترسی راحت
# -------------------------------
# Download dataset1
# -------------------------------
import gdown
import os

os.makedirs("data", exist_ok=True)

file_id = "1RmwsUdK-k3nSTUL97sKn9V0RJctG6KLy"
url = f"https://drive.google.com/uc?id={file_id}"  # Direct download link

out_path = "data/movie.csv"

print("Downloading dataset...")
gdown.download(url, out_path, quiet=False)
print(f"✅ Dataset downloaded to {out_path}")

movie = out_path


# -------------------------------
# Download dataset2
# -------------------------------
os.makedirs("data", exist_ok=True)

file_id = "1cdChLBc9J_T-c7DjROie42AER3-7-0H9"
url = f"https://drive.google.com/uc?id={file_id}"  # Direct download link

out_path = "data/rate.csv"

print("Downloading dataset...")
gdown.download(url, out_path, quiet=False)
print(f"✅ Dataset downloaded to {out_path}")

rate = out_path

Downloading...
From: https://drive.google.com/uc?id=1RmwsUdK-k3nSTUL97sKn9V0RJctG6KLy
To: /content/data/movie.csv
100%|██████████| 494k/494k [00:00<00:00, 7.28MB/s]


✅ Dataset downloaded to data/movie.csv


Downloading...
From: https://drive.google.com/uc?id=1cdChLBc9J_T-c7DjROie42AER3-7-0H9
To: /content/data/rate.csv
100%|██████████| 2.48M/2.48M [00:00<00:00, 20.7MB/s]

✅ Dataset downloaded to data/rate.csv


In [5]:
# بارگذاری دیتا ست اول که مربوط لیست فیلم ها و مشخصه آنهاست به وسیله pandas
movies_data = pd.read_csv(movie)

#بارگذاری دیتا ست دوم که مربوط به امتیاز دهی کاربران به فیلم هاست توسط pandas
ratings_data = pd.read_csv(rate)

# --- پردازش ستون‌های فیلم‌ها --

In [6]:
#مشاهده 5 خط اول داده فیلم ها
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# میبینیم که سال ساخت فیلم ها جلوی اسمشون نوشته شده که مشکل داره این شکل :
# سال ساخت غیر قابل استفاده شده

In [8]:
#جدا کردن اسم فیلم ها و تارخ از هم:

#اول میایم سال ساخت رو با پرانتزش بر میریزیم توی ستونی به نام year
movies_data['year'] = movies_data['title'].str.extract(r'\((\d{4})\)', expand=False)

#حالا پرانتر های اونهارو بر میداریم
movies_data['year'] = movies_data.year.str.extract('(\d\d\d\d)',expand=False)

#حالا تاریخ رو از جلوی اسم فیلم ها در ستون title بر میداریم
movies_data['title'] = movies_data['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)

#فضای خالی که جلوی اسم ها تشکیل شدن رو حذف میکنیم
movies_data['title'] = movies_data['title'].apply(lambda x: x.strip())

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-201005956.py:7: SyntaxWarning: invalid escape sequence '\d'
  movies_data['year'] = movies_data.year.str.extract('(\d\d\d\d)',expand=False)


In [9]:
#ببیتیم چیکار کردیم تا اینجا
movies_data.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [10]:
# ستون ژانر رو پاک میکنیم چرا که نیازی بهش نداریم
# چون سیتسم ما یه این شکله که از یوزر های شبیه به شما به شما پیشنهاد میده نه از تاپیک های شما
movies_data = movies_data.drop('genres', axis=1)

In [11]:
# دیتا نهایی ما به این شکل شد
movies_data.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


# --- پردازش امتیاز کاربران ---

In [12]:
#مشاهده 5 خط اول داده داده امتیاز کاربران
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
# خب این دیتا فریم که میبینید به این شکله که :
# userid: آیدی یوزری هست که امتیاز داده
# movieid: اومده فیلم هارو براشون آیدی در نظرگرفته که شکل نوشتن فیلم ها روی دیتا تاثیر نداشته باشه
# rating : اون امتیازی که کاربر داده
#timestamp: زمانیه که اون کاربر نظر داده که

In [14]:
# حذف ستون timestamp
ratings_data = ratings_data.drop('timestamp', axis=1)

In [15]:
# دیتا ما در نهایت یچیز به این شکل شده:
ratings_data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [16]:
## همونطور که گفتم قراره یه سیتسم پیشنهاد دهی طراحی کنیم که به آن کاربر به کاربر هم میگن
## این سیستم از نظرات کاربران دیگه استفاده میکنه و به کاربر جدیدی که با اون کاربر ها نقطه مشترکی داره پیشنهاد میده

## این تکنیک تلاش می‌کند کاربرانی را پیدا کند که نظرات مشابهی با شخص جدید دارند
## و سپس آیتم‌هایی را که آنها دوست داشته‌اند به شخص جدید پیشنهاد می‌دهد
##. روش‌های مختلفی برای یافتن کاربران مشابه وجود دار .. روشی که ما در اینجا استفاده خواهیم کرد، مبتنی بر تابع همبستگی پیرسون خواهد بود

## خب بیاید یه شخض جدید وارد کنیم

#امتیاز هایی که این شخص به 5 فیلم داده رو داریم

# --- تعریف کاربر جدید برای سیستم توصیه ---
NewUser = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ]

#مشخصات شخص رو به شکل دیتافریم نمایش بدیم که بهتر متوجه بشیم
inputMovies = pd.DataFrame(NewUser)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [17]:
# حالا از اون دیتا ستی که همه فیلم هامون توشه
# هر فیلمی که تایتلشون شبیه به تایتل یوزر جدیدمون هست رو جدا میکنیم
inputId = movies_data[movies_data['title'].isin(inputMovies['title'].tolist())]

inputId

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
257,296,Pulp Fiction,1994
973,1274,Akira,1988
1445,1968,"Breakfast Club, The",1985


In [18]:
## حالا امتیاز کاربرمون رو اضافه میکنیم به این دیتاستمون
inputMovies = pd.merge(inputId, inputMovies)

In [19]:
# از دیتامون ستون هایی که نمیخوایم رو پاک میکنیم مثل سال
inputMovies = inputMovies.drop('year', axis = 1)

In [20]:
#رخ نهایی دیتامون
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [21]:
# خب الان کاربرانی رو که این فیلم هارو دیدن رو از یقبه جدا میکتیم
userSubset = ratings_data[ratings_data['movieId'].isin(inputMovies['movieId'].tolist())]

userSubset.head()

,userId,movieId,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0


In [22]:
 # حالا میایم و گروه بندی میکنیم داده هامونو بر اساس آیدی یوزر هامون
userSubsetGroup = userSubset.groupby(['userId'])
userSubsetGroup.head()



,userId,movieId,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0
...,...,...,...
99510,609,296,4.0
99534,610,1,5.0
99552,610,296,5.0
99636,610,1274,5.0


:مثلا فرض کن یه دیتافریم مثل این داریم
| userId | movieId | rating |
| ------ | ------- | ------ |
| 1      | 10      | 4.0    |
| 1      | 15      | 5.0    |
| 2      | 10      | 3.0    |
| 2      | 20      | 2.0    |

: شما این جدول رو به دو گروه تقسیم می‌کنی

یک گروه شامل همه ردیف‌هایی که userId == 1

یک گروه شامل همه ردیف‌هایی که userId == 2

In [23]:
# محاسبه تعداد فیلم‌هایی که هر کاربر امتیاز داده
userSubsetGroup.size()

,0
userId,
1,2
4,2
5,2
6,2
7,1
...,...
606,4
607,3
608,5


In [24]:
# برای افزایش دقت توصیه‌ها، کاربران را بر اساس تعداد فیلم‌هایی که با کاربر ورودی به‌طور مشترک مشاهده کرده‌اند، اولویت‌بندی می‌کنیم

#این کار باعث می‌شود به جای مقایسه با تمام کاربران، تمرکز روی نزدیک‌ترین کاربران از نظر علایق باشد و همه کاربران رو چک نکنیم


In [25]:
# کاربران را طوری مرتب می‌کنیم که آن‌هایی که بیشترین تعداد فیلم مشترک با کاربر ورودی را دارند، در اولویت بالاتری قرار گیرند
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [26]:
#خب بیاین 5 تا یوزر اول رو ببینیم
#میبینیم چه فیلمی دیدن و چه نمره ای دادن
userSubsetGroup[0:5]

[((91,),
         userId  movieId  rating
  14121      91        1     4.0
  14122      91        2     3.0
  14173      91      296     4.5
  14316      91     1274     5.0
  14383      91     1968     3.0),
 ((177,),
         userId  movieId  rating
  24900     177        1     5.0
  24901     177        2     3.5
  24930     177      296     5.0
  25069     177     1274     2.0
  25129     177     1968     3.5),
 ((219,),
         userId  movieId  rating
  31524     219        1     3.5
  31525     219        2     2.5
  31554     219      296     4.0
  31628     219     1274     2.5
  31680     219     1968     3.0),
 ((274,),
         userId  movieId  rating
  39229     274        1     4.0
  39230     274        2     3.5
  39288     274      296     5.0
  39448     274     1274     4.0
  39549     274     1968     4.0),
 ((298,),
         userId  movieId  rating
  44535     298        1     2.0
  44536     298        2     0.5
  44555     298      296     4.5
  44620     298    

### شباهت یابی

در این مرحله می‌خوایم کاربرانی که بیشترین شباهت رو به کاربر ورودی دارن پیدا کنیم
برای اینکه کار سریع‌تر باشه، فقط یه گروه کوچیک از کاربران رو بررسی می‌کنیم

برای سنجش شباهت، از ضریب همبستگی پیرسون استفاده می‌کنیم.

چرا پیرسون؟
این ضریب نشون می‌ده چقدر امتیازهای دو کاربر به هم شبیهن، حتی اگه یه کاربر همیشه امتیازها رو بزرگ‌ تر یا کوچیک‌تر بده


1 یعنی سلیقه‌ها خیلی شبیهن

0 یعنی هیچ شباهتی نیست

-1 یعنی سلیقه‌ها کاملا متفاوتن

In [27]:
#برای کاهش پیچیدگی محاسبات و تمرکز بر کاربران با بیشترین شباهت، تنها ۱۰۰ کاربر اول از گروه‌بندی مرتب‌شده را انتخاب می‌کنیم
userSubsetGroup = userSubsetGroup[0:100]

In [28]:
#حالا می‌ریم شباهت کاربر ورودی رو با هر کاربر زیرمجموعه رو با پیرسون حساب کنیم
#و تو یه دیکشنری ذخیره کنیم که شماره کاربر رو کلید و مقدار شباهتش رو مقدار بذاریم

In [29]:
# ساختن یک دیکشنری خالی برای ذخیره ضریب همبستگی پیرسون
# کلید دیکشنری شماره کاربر و مقدارش مقدار ضریب هست
pearsonCorrelationDict = {}

# شروع حلقه روی هر گروه کاربری در زیرمجموعه (userSubsetGroup)
# name شناسه کاربر و group داده‌های اون کاربر هستند
for name, group in userSubsetGroup:

    # مرتب کردن داده‌ها بر اساس شناسه فیلم (movieId)
    # این کار باعث میشه داده‌ها برای مقایسه درست کنار هم باشن
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')

    # پیدا کردن تعداد فیلم‌هایی که این کاربر دیده (N)
    nRatings = len(group)

    # گرفتن فیلم‌هایی که هر دو کاربر (ورودی و فعلی) دیده‌اند
    # این قسمت فیلم‌های مشترک بین inputMovies و group رو فیلتر می‌کنه
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]

    # گرفتن امتیازهای کاربر ورودی برای فیلم‌های مشترک به صورت لیست
    tempRatingList = temp_df['rating'].tolist()

    # گرفتن امتیازهای کاربر فعلی برای فیلم‌های مشترک به صورت لیست
    tempGroupList = group['rating'].tolist()

    # محاسبه اجزای فرمول پیرسون:
    #مجموع مربعات امتیازهای کاربر ورودی منهای مجذور مجموع امتیازها تقسیم بر N
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList), 2) / float(nRatings)

    # Syy برای کاربر فعلی به همین صورت
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList), 2) / float(nRatings)

    # حاصل جمع ضرب امتیازهای دو کاربر منهای حاصل ضرب مجموع امتیازها تقسیم بر N
    Sxy = sum(i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList) * sum(tempGroupList) / float(nRatings)

    # اگر مخرج کسر صفر نبود، ضریب پیرسون رو حساب می‌کنیم
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy / sqrt(Sxx * Syy)
    else:
        # اگه مخرج صفر بود یعنی نمی‌تونیم ضریب رو حساب کنیم، ضریب رو صفر قرار میدیم
        pearsonCorrelationDict[name] = 0



In [30]:
## الان همه کاربران و میزان شباهتشونو داریم
pearsonCorrelationDict.items()

dict_items([((91,), 0.43852900965351443), ((177,), 0.0), ((219,), 0.45124262819713973), ((274,), 0.716114874039432), ((298,), 0.9592712306918567), ((414,), 0.9376144618769914), ((474,), 0.11720180773462392), ((477,), 0.4385290096535153), ((480,), 0.7844645405527362), ((483,), 0.08006407690254357), ((599,), 0.7666866491579839), ((608,), 0.920736884379251), ((50,), 0.15713484026367722), ((57,), -0.7385489458759964), ((68,), 0.0), ((103,), 0.5222329678670935), ((135,), 0.8703882797784892), ((182,), 0.9428090415820635), ((202,), 0.5222329678670935), ((217,), 0.30151134457776363), ((226,), 0.9438798074485389), ((288,), 0.6005325641789633), ((307,), 0.9655810287305759), ((318,), 0.44486512077567225), ((322,), 0.5057805388588731), ((330,), 0.9035942578600878), ((357,), 0.5606119105813882), ((434,), 0.9864036607532465), ((448,), 0.30151134457776363), ((469,), 0.8164965809277261), ((561,), 0.5222329678670935), ((600,), 0.18442777839082938), ((606,), 0.9146591207600472), ((610,), -0.471404520791

In [31]:
# # با اینکار راحت تر کد بالارو متوجه میشین که توش چیه
# # هر یوزر و مقدار شباهتش جلوش
for userId, correlation in pearsonCorrelationDict.items():
    print(f"User {userId} similarity: {correlation}")

User (91,) similarity: 0.43852900965351443
User (177,) similarity: 0.0
User (219,) similarity: 0.45124262819713973
User (274,) similarity: 0.716114874039432
User (298,) similarity: 0.9592712306918567
User (414,) similarity: 0.9376144618769914
User (474,) similarity: 0.11720180773462392
User (477,) similarity: 0.4385290096535153
User (480,) similarity: 0.7844645405527362
User (483,) similarity: 0.08006407690254357
User (599,) similarity: 0.7666866491579839
User (608,) similarity: 0.920736884379251
User (50,) similarity: 0.15713484026367722
User (57,) similarity: -0.7385489458759964
User (68,) similarity: 0.0
User (103,) similarity: 0.5222329678670935
User (135,) similarity: 0.8703882797784892
User (182,) similarity: 0.9428090415820635
User (202,) similarity: 0.5222329678670935
User (217,) similarity: 0.30151134457776363
User (226,) similarity: 0.9438798074485389
User (288,) similarity: 0.6005325641789633
User (307,) similarity: 0.9655810287305759
User (318,) similarity: 0.44486512077567

In [32]:
#خیلی زشته هنوز نه؟
#بریم یه دیتا فریم ازش بسازیم

In [33]:
#مقدار ضریب همبستگی رو به دیکشنری تبدیل میکنیم

# اول کلیدهای تاپل رو به عدد تبدیل می‌کنیم (اولین عضو تاپل)
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')

#دیتا بالارو به دیکشنری تبدیل مینیم جوریکه شناسه کاربر ها هم ردیف شه
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')

#یه ستون میسازیم براش که میزان شباهتا توش باشه
pearsonDF.columns = ['similarityIndex']

#یه ستون دیگه میسازیم برای آیدی کاربران
pearsonDF['userId'] = pearsonDF.index



# فقط اولین عضو هر تاپل رو می‌گیریم
pearsonDF['userId'] = pearsonDF.index.map(lambda x: x[0] if isinstance(x, tuple) else x)
#اسم ردیف ستون هارو از صفر شماره میزاریم به بالا
pearsonDF.index = range(len(pearsonDF))

#حالا 5 تا اولشو میریم ببینیم
pearsonDF.head()


,similarityIndex,userId
0,0.438529,91
1,0.000000,177
2,0.451243,219
3,0.716115,274
4,0.959271,298


In [34]:
# دیکشنری برای ذخیره ضریب همبستگی پیرسون هر کاربر نسبت به کاربر ورودی
pearsonCorrelationDict = {}

# برای هر گروه کاربری در زیرمجموعه کاربران
for userId, group in userSubsetGroup:
    # مرتب‌سازی گروه فعلی و داده‌های ورودی بر اساس شناسه فیلم برای جلوگیری از اشتباه در مقایسه
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')

    # تعداد فیلم‌هایی که این کاربر دیده است
    nRatings = len(group)

    # گرفتن امتیازهای فیلم‌هایی که هر دو کاربر (ورودی و این گروه) مشترکاً دیده‌اند
    commonMovies = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]

    # امتیازهای کاربر ورودی برای فیلم‌های مشترک، به صورت لیست
    inputRatings = commonMovies['rating'].tolist()

    # امتیازهای کاربر فعلی برای فیلم‌های مشترک، به صورت لیست
    groupRatings = group['rating'].tolist()

    # محاسبه اجزای فرمول همبستگی پیرسون:
    Sxx = sum([i**2 for i in inputRatings]) - (sum(inputRatings)**2) / nRatings
    Syy = sum([i**2 for i in groupRatings]) - (sum(groupRatings)**2) / nRatings
    Sxy = sum(i*j for i, j in zip(inputRatings, groupRatings)) - (sum(inputRatings)*sum(groupRatings)) / nRatings

    # اگر مخرج فرمول صفر نشود، ضریب پیرسون را محاسبه کن؛ در غیر اینصورت مقدار 0 بده
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[userId] = Sxy / sqrt(Sxx * Syy)
    else:
        pearsonCorrelationDict[userId] = 0

# نمایش 5 ردیف اول دیتافریم
pearsonDF.head()


,similarityIndex,userId
0,0.438529,91
1,0.000000,177
2,0.451243,219
3,0.716115,274
4,0.959271,298


In [35]:
#حالا می‌خوایم ۵۰ کاربری که بیشترین شباهت رو به کاربر ورودی دارن پیدا کنیم

In [36]:
#مرتب میکنیم بر اساس بیشترین تشابه به کمترین
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]

#انتظار میره دیگه بدونین  این چیه
topUsers.head()

,similarityIndex,userId
43,1.0,132
34,1.0,18
46,1.0,144
63,1.0,305
82,1.0,489


In [37]:
#حالا می‌خوایم به کاربر فیلم پیشنهاد بدیم
#برای این کار، امتیازهای بقیه کاربرا رو می‌گیریم و با توجه به شباهتی که دارن (همون ضریب پیرسون)، میانگین وزن‌دارش رو حساب می‌کنیم

#اول باید فیلم‌هایی که این کاربران دیدن رو از جدول امتیازها برداریم و شباهت هر کاربر رو کنار امتیازها بذاریم
#این کار با ترکیب دو جدول انجام می‌شه

In [38]:
topUsersRating=topUsers.merge(ratings_data, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,132,1,2.0
1,1.0,132,17,3.0
2,1.0,132,29,2.0
3,1.0,132,32,3.0
4,1.0,132,34,1.5


In [39]:
#حالا تنها کاری که باید انجام دهیم این است که امتیاز فیلم را در وزن آن (شاخص شباهت) ضرب کنیم،
#سپس امتیازهای جدید را جمع کرده و بر مجموع وزن‌ها تقسیم کنیم

#حالا امتیاز فیلم رو تو اون شاخص شباهتی که بدست آوردیم ضرب میکنیم و بر مجموع تقسیم میکنیم
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,132,1,2.0,2.0
1,1.0,132,17,3.0,3.0
2,1.0,132,29,2.0,2.0
3,1.0,132,32,3.0,3.0
4,1.0,132,34,1.5,1.5


In [40]:
#ابتدا دیتا ست بالامون رو بر اساس اسم فیلم ها گروه بندی میکنیم
#یعنی برای هر فیبم کاربرهای مختلف که نمره دادن رو تو یه گروه قرار میدیم
#سپس از هر گروه دو ستون را محاسبه میکنیم
# similarityIndex --> جمع شباهت کاربرها برای اون فیلم
# weightedRating -> جمع نمره وزن دار شده یعنی (نمره * شباهت)
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]

#حال اسم ستون هارو میزاریم
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']

# 5تا اول
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,36.354096,133.167946
2,31.005292,94.904257
3,8.783859,26.381456
4,0.866025,1.732051
5,7.165336,19.775255


مثلا اگر دو کاربر مشابه فیلم 42 رو اینجوری نمره داده باشن:

| userId | movieId  | rating | similarityIndex | weightedRating  |
| ------ | ------- | ------ |------ |------ |
| 1      | 42      | 5       |0.9   |4.5    |
| 3      | 42      | 4   |0.7    |2.8   |

بعد از اجرای groupby و sum::


| movieId  | sum_similarityIndex | sum_weightedRating |
| ------- | ------ |------
| 42      | 0.9 + 0.7 = 1.6       |4.5 + 2.8 = 7.3   |


In [41]:
#یه دیتا فریم خالی میسازیم
recommendation_df = pd.DataFrame()

#حالا میانگین وزن هارو حساب میکنیم و میریزیم تو دیتا فریم خالیمون

##
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']

#آیدی فیلم هارو اضافه میکنیم بهش جوری که جزوی از دیتا فریممون نیاشه ولی کنارش نمایش بده
recommendation_df['movieId'] = tempTopUsersRating.index

recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.663080,1
2,3.060905,2
3,3.003402,3
4,2.000000,4
5,2.759850,5


In [42]:
# حالا مرتبش میکنیم و 20 فیلم اولی که لیگوریتم ما پیشنهاد داده رو میبینیم

In [43]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(20)

,weighted average recommendation score,movieId
movieId,,
905,5.0,905
7579,5.0,7579
3310,5.0,3310
1211,5.0,1211
167064,5.0,167064
187593,5.0,187593
3951,5.0,3951
158027,5.0,158027
160644,5.0,160644


In [44]:
#حالا به کمک آیدی فیلم ها مشخصات کامل فیلم رو از دیتای فیلم هامون میبنیم

In [45]:
movies_data.loc[movies_data['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
687,905,It Happened One Night,1934
912,1211,"Wings of Desire (Himmel über Berlin, Der)",1987
2484,3310,"Kid, The",1921
2947,3951,Two Family House,2000
4969,7579,Pride and Prejudice,1940
9284,158027,SORI: Voice from the Heart,2016
9337,160644,Indignation,2016
9367,162414,Moonlight,NaN
9443,167064,I Am Not Your Negro,2017
9709,187593,Deadpool 2,2018


In [46]:
# soroosh taghados